In [1]:
import pandas as pd


loc_meta_df = pd.read_csv("data/clark-county-rain-gauges/CCRFCD Station Locations 2025.csv")
prev_meta_df = pd.read_csv("data/clark-county-rain-gauges/clark_county_gauge_metadata.csv")

In [2]:
loc_meta_df.head()

,Station ID,Name,Type,OOS,Latitude,Longitude
0,2,Willow Beach 2 (NPS),NPS System,False,35.87789,-114.58875
1,4,Malpais Flattop Mesa (NPS),NPS System,False,35.82014,-114.66325
2,5,Householder Pass (NPS),NPS System,False,35.82578,-114.59625
3,7,Eldorado (NPS) - New Repeater,NPS System,False,35.74642,-114.57600
4,8,Willow Beach 8 (NPS),NPS System,False,35.96100,-114.62764


In [3]:
prev_meta_df = prev_meta_df.drop("Unnamed: 0", axis=1)
prev_meta_df = prev_meta_df.rename(
    columns={
        "gauge_number": "Station ID",
        "gauge_name"  : "old_name"
    }
)
prev_meta_df.head()

,Station ID,old_name
0,1,Willow Beach 1 (NPS) RAIN (old) (Rain)
1,2,Willow Beach 2 (NPS) RAIN (Rain)
2,3,Willow Beach 3 (NPS) RAIN (Rain)
3,4,Malpais Flattop Mesa (NPS) RAIN (Rain)
4,5,Householder Pass (NPS) RAIN (Rain)


In [4]:
station_ids_map = {}
for row in loc_meta_df.iterrows():
    _row = row[1]
    station_id = _row["Station ID"]
    station_ids_map[station_id] =  _row

In [5]:
old_names   = []
station_ids = []
names       = []
types       = []
ooss        = []
lats        = []
lons        = []


for row in prev_meta_df.iterrows():

    old_name    = row[1]['old_name']
    _station_id = row[1]['Station ID']

    if _station_id not in station_ids_map: 
        station_id = None
        name       = None
        type       = None
        oos        = None
        lat        = None
        lon        = None
    else:
        _row = station_ids_map[_station_id]
        station_id = _row["Station ID"]
        name       = _row["Name"]
        type       = _row["Type"]
        oos        = _row["OOS"]
        lat        = _row["Latitude"]
        lon        = _row["Longitude"]
    
    old_names.append(old_name)
    station_ids.append(station_id)
    names.append(name)
    types.append(type)
    ooss.append(oos)
    lats.append(lat)
    lons.append(lon)


In [6]:
from pandas import DataFrame


metadf = DataFrame({
    "station_id": station_ids,
    "name": names,
    "old_name": old_names,
    "type": types,
    "oos": ooss,
    "lat": lats,
    "lon": lons,
})

metadf.to_csv("ccrfcd_rain_gauge_metadata.csv")

# **merge all precip data**
***

In [11]:
import pandas as pd
from pathlib import Path


metadata_fp     = "data/clark-county-rain-gauges/ccrfcd_rain_gauge_metadata.csv"
rain_gauge_dir  = "data/clark-county-rain-gauges/2021-"
rain_gauge_csvs = [f for f in Path(rain_gauge_dir).glob("*.csv")]
metadata        = pd.read_csv(metadata_fp)

In [12]:
# remove invalid gauges
metadata         = metadata[metadata["station_id"] > 0]
unique_gauge_ids = sorted(list(set(metadata['station_id'].astype(int))))

In [13]:
valid_unique_gauge_ids = set()

for id in unique_gauge_ids:
    gaugedata_fp = Path(rain_gauge_dir) / Path(f"gagedata_{id}.csv")
    if gaugedata_fp.is_file():
        valid_unique_gauge_ids.add(id)
    else:
        print(gaugedata_fp)
        break

unique_gauge_ids.__len__(), valid_unique_gauge_ids.__len__()

data/clark-county-rain-gauges/2021-/gagedata_2584.csv


(231, 11)

In [14]:
"""
Helper methods to scrape rain-gauge data from:
https://gustfront.ccrfcd.org/gagedatalist/
"""

import time

from pathlib import Path
from datetime import datetime
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select, WebDriverWait


_URL = "https://gustfront.ccrfcd.org/gagedatalist/"


def get_chrome_driver() -> webdriver.Chrome:

    options = Options()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--headless")
    curr_dir = "/playpen/mufan/levi/tianlong-chen-lab/nws-lv-precip-forecasting/data/clark-county-rain-gauges/_test"
    prefs = {
        "download.default_directory": curr_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
    }
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options=options)
    driver.execute_cdp_cmd(
        "Page.setDownloadBehavior",
        {"behavior": "allow", "downloadPath": curr_dir},
    )
    return driver


driver = get_chrome_driver()
driver.get(_URL)
print(driver.title)

gauge_element     = driver.find_element(By.ID, "uiGage")
gauge_selector    = Select(gauge_element)
gauge_selector.select_by_index(0)

gauges            = gauge_element.find_elements(By.TAG_NAME, "option")
all_gauge_names   = [g.accessible_name for g in gauges]
total_gauges      = len(all_gauge_names)
gauge_id_name_map = {}

for name in all_gauge_names:
    gauge_id = int(name.split(" - ")[0])
    gauge_id_name_map[gauge_id] = name

download_element = driver.find_element(By.ID, "uiDownload")

Regional Flood Control District - Gage Data


In [15]:
for gauge_id in tqdm(unique_gauge_ids, total=len(unique_gauge_ids)):

    visible_text = gauge_id_name_map[gauge_id]

    try:
        
        gauge_element = driver.find_element(By.ID, "uiGage")
        gauge_selector = Select(gauge_element)
        gauge_selector.select_by_visible_text(visible_text)

        # 2. set start/end time
        start_date_element = driver.find_element(By.ID, "startDate")
        start_date_element.send_keys(Keys.CONTROL + "a")

        # 2a. set start date
        start_date = datetime(2021, 1, 1)
        start_date_element.send_keys(start_date.strftime("%m/%d/%Y"))
        end_date_element = driver.find_element(By.ID, "endDate")

        # 3. set interval to second index value
        interval_element = driver.find_element(By.ID, "uiInterval")
        interval_select = Select(interval_element)

        # 3a. set interval to "5 minutes"
        interval_select.select_by_index(1)
        
        download_element.click()

    except Exception as e:

        driver.quit()
        driver = get_chrome_driver()
        driver.get(_URL)

        print(e)
        print(f"failed to download {visible_text}")
        driver.quit()
        driver = get_chrome_driver()
        driver.get(_URL)

        gauge_element = driver.find_element(By.ID, "uiGage")
        gauge_selector = Select(gauge_element)
        gauge_selector.select_by_visible_text(visible_text)

        # 2. set start/end time
        start_date_element = driver.find_element(By.ID, "startDate")
        start_date_element.send_keys(Keys.CONTROL + "a")

        # 2a. set start date
        start_date = datetime(2021, 1, 1)
        start_date_element.send_keys(start_date.strftime("%m/%d/%Y"))
        end_date_element = driver.find_element(By.ID, "endDate")

        # 3. set interval to second index value
        interval_element = driver.find_element(By.ID, "uiInterval")
        interval_select = Select(interval_element)

        # 3a. set interval to "5 minutes"
        interval_select.select_by_index(1)

        # 4. click "Download CSV File" and collect result
        download_element = driver.find_element(By.ID, "uiDownload")
        download_element.click()


driver.quit()

  0%|          | 0/231 [00:00<?, ?it/s]

  5%|▌         | 12/231 [00:59<45:22, 12.43s/it]

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="uiGage"]"}
  (Session info: chrome=137.0.7151.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5dfca3ff645a <unknown>
#1 0x5dfca3a9b760 <unknown>
#2 0x5dfca3aed127 <unknown>
#3 0x5dfca3aed321 <unknown>
#4 0x5dfca3b3be94 <unknown>
#5 0x5dfca3b12e7d <unknown>
#6 0x5dfca3b3930b <unknown>
#7 0x5dfca3b12c23 <unknown>
#8 0x5dfca3adf4a5 <unknown>
#9 0x5dfca3ae0111 <unknown>
#10 0x5dfca3fbaf1b <unknown>
#11 0x5dfca3fbee19 <unknown>
#12 0x5dfca3fa1ac9 <unknown>
#13 0x5dfca3fbf9c8 <unknown>
#14 0x5dfca3f8634f <unknown>
#15 0x5dfca3fe3a28 <unknown>
#16 0x5dfca3fe3c06 <unknown>
#17 0x5dfca3ff5336 <unknown>
#18 0x702ab5e9caa4 <unknown>
#19 0x702ab5f29c3c <unknown>

failed to download 2594 - Town Wash DB RAIN (Rain)


  6%|▋         | 15/231 [01:43<43:18, 12.03s/it]  

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="uiGage"]"}
  (Session info: chrome=137.0.7151.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5eed49fd145a <unknown>
#1 0x5eed49a76760 <unknown>
#2 0x5eed49ac8127 <unknown>
#3 0x5eed49ac8321 <unknown>
#4 0x5eed49b16e94 <unknown>
#5 0x5eed49aede7d <unknown>
#6 0x5eed49b1430b <unknown>
#7 0x5eed49aedc23 <unknown>
#8 0x5eed49aba4a5 <unknown>
#9 0x5eed49abb111 <unknown>
#10 0x5eed49f95f1b <unknown>
#11 0x5eed49f99e19 <unknown>
#12 0x5eed49f7cac9 <unknown>
#13 0x5eed49f9a9c8 <unknown>
#14 0x5eed49f6134f <unknown>
#15 0x5eed49fbea28 <unknown>
#16 0x5eed49fbec06 <unknown>
#17 0x5eed49fd0336 <unknown>
#18 0x75a47129caa4 <unknown>
#19 0x75a471329c3c <unknown>

failed to download 2674 - Pulsipher Wash DB RAIN (Rain)


...

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="uiGage"]"}
  (Session info: chrome=137.0.7151.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5744de23145a <unknown>
#1 0x5744ddcd6760 <unknown>
#2 0x5744ddd28127 <unknown>
#3 0x5744ddd28321 <unknown>
#4 0x5744ddd76e94 <unknown>
#5 0x5744ddd4de7d <unknown>
#6 0x5744ddd7430b <unknown>
#7 0x5744ddd4dc23 <unknown>
#8 0x5744ddd1a4a5 <unknown>
#9 0x5744ddd1b111 <unknown>
#10 0x5744de1f5f1b <unknown>
#11 0x5744de1f9e19 <unknown>
#12 0x5744de1dcac9 <unknown>
#13 0x5744de1fa9c8 <unknown>
#14 0x5744de1c134f <unknown>
#15 0x5744de21ea28 <unknown>
#16 0x5744de21ec06 <unknown>
#17 0x5744de230336 <unknown>
#18 0x71471389caa4 <unknown>
#19 0x714713929c3c <unknown>

failed to download 3129 - Moapa SW 2 RAIN (Rain)


 11%|█▏        | 26/231 [05:58<1:18:06, 22.86s/it]

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="uiGage"]"}
  (Session info: chrome=137.0.7151.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x636da970545a <unknown>
#1 0x636da91aa760 <unknown>
#2 0x636da91fc127 <unknown>
#3 0x636da91fc321 <unknown>
#4 0x636da924ae94 <unknown>
#5 0x636da9221e7d <unknown>
#6 0x636da924830b <unknown>
#7 0x636da9221c23 <unknown>
#8 0x636da91ee4a5 <unknown>
#9 0x636da91ef111 <unknown>
#10 0x636da96c9f1b <unknown>
#11 0x636da96cde19 <unknown>
#12 0x636da96b0ac9 <unknown>
#13 0x636da96ce9c8 <unknown>
#14 0x636da969534f <unknown>
#15 0x636da96f2a28 <unknown>
#16 0x636da96f2c06 <unknown>
#17 0x636da9704336 <unknown>
#18 0x7c6e9a89caa4 <unknown>
#19 0x7c6e9a929c3c <unknown>

failed to download 3134 - Moapa SW 1 RAIN (Rain)


 11%|█▏        | 26/231 [06:30<51:15, 15.00s/it]  


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="uiGage"]"}
  (Session info: chrome=137.0.7151.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x63e833d8045a <unknown>
#1 0x63e833825760 <unknown>
#2 0x63e833877127 <unknown>
#3 0x63e833877321 <unknown>
#4 0x63e8338c5e94 <unknown>
#5 0x63e83389ce7d <unknown>
#6 0x63e8338c330b <unknown>
#7 0x63e83389cc23 <unknown>
#8 0x63e8338694a5 <unknown>
#9 0x63e83386a111 <unknown>
#10 0x63e833d44f1b <unknown>
#11 0x63e833d48e19 <unknown>
#12 0x63e833d2bac9 <unknown>
#13 0x63e833d499c8 <unknown>
#14 0x63e833d1034f <unknown>
#15 0x63e833d6da28 <unknown>
#16 0x63e833d6dc06 <unknown>
#17 0x63e833d7f336 <unknown>
#18 0x731a6709caa4 <unknown>
#19 0x731a67129c3c <unknown>


In [ ]:
column_headers   = ["time"] + sorted(list(valid_unique_gauge_ids))
column_headers